# Caso Final 4

In [1]:

%pylab
%matplotlib inline

%config InlineBackend.figure_format = 'retina'

import numpy as np

Using matplotlib backend: GTK3Agg
Populating the interactive namespace from numpy and matplotlib


# POS tagger training para el corpues en español.

In [3]:
import nltk
from nltk.corpus import cess_esp

sents = cess_esp.tagged_sents()
print(sents)

[[('El', 'da0ms0'), ('grupo', 'ncms000'), ('estatal', 'aq0cs0'), ('Electricité_de_France', 'np00000'), ('-Fpa-', 'Fpa'), ('EDF', 'np00000'), ('-Fpt-', 'Fpt'), ('anunció', 'vmis3s0'), ('hoy', 'rg'), (',', 'Fc'), ('jueves', 'W'), (',', 'Fc'), ('la', 'da0fs0'), ('compra', 'ncfs000'), ('del', 'spcms'), ('51_por_ciento', 'Zp'), ('de', 'sps00'), ('la', 'da0fs0'), ('empresa', 'ncfs000'), ('mexicana', 'aq0fs0'), ('Electricidad_Águila_de_Altamira', 'np00000'), ('-Fpa-', 'Fpa'), ('EAA', 'np00000'), ('-Fpt-', 'Fpt'), (',', 'Fc'), ('creada', 'aq0fsp'), ('por', 'sps00'), ('el', 'da0ms0'), ('japonés', 'aq0ms0'), ('Mitsubishi_Corporation', 'np00000'), ('para', 'sps00'), ('poner_en_marcha', 'vmn0000'), ('una', 'di0fs0'), ('central', 'ncfs000'), ('de', 'sps00'), ('gas', 'ncms000'), ('de', 'sps00'), ('495', 'Z'), ('megavatios', 'ncmp000'), ('.', 'Fp')], [('Una', 'di0fs0'), ('portavoz', 'nccs000'), ('de', 'sps00'), ('EDF', 'np00000'), ('explicó', 'vmis3s0'), ('a', 'sps00'), ('EFE', 'np00000'), ('que', 'c

En la practica 2 del modulo hemos comprobado el tagger mas completo
, asi que para este caso final se va a usar HiddenMarkovModelTagger.Para entrenar el POS tagger vamos a dividir los datos en entrenamiento y en test. El 90% sera de entrenamiento y el 10% sera con el que evaluemos nuestro tagger.

In [4]:
training = []
testing = []

for i in range(len(sents)) :
    if i % 10 :
        training.append(sents[i])
    else :
        testing.append(sents[i])
        
from nltk.tag.hmm import HiddenMarkovModelTagger
spanish_pos_tagger = HiddenMarkovModelTagger.train(training)

print('Evaluation:', spanish_pos_tagger.evaluate(testing)*100)

Evaluation: 89.88905831011094


Usaremos MarkovMOdel tagger para nuestro analisi morfologico ya que no se ve sobreajustado y la evaluacion es alta.


# Pos tagset para el corpus en español

Observamos todas las etiquetas que nos porporciona nuestro tagger para el español.

In [10]:
sentence = 'yo quiero una pizza de cuatro quesos, y tres hamburguesas con patatas'

tokens = nltk.word_tokenize(sentence)

tagged = spanish_pos_tagger.tag(tokens)
tagged

[('yo', 'pp1csn00'),
 ('muchas', 'rn'),
 ('quiero', 'vmip1s0'),
 ('una', 'di0fs0'),
 ('pizza', 'ncfs000'),
 ('de', 'sps00'),
 ('cuatro', 'dn0cp0'),
 ('quesos', 'ncmp000'),
 (',', 'Fc'),
 ('y', 'cc'),
 ('tres', 'dn0cp0'),
 ('hamburguesas', 'ncmp000'),
 ('con', 'sps00'),
 ('patatas', 'ncfp000')]

Ahora podemos obeservar las etiquetas usadas y crear la gramatica para el regexparser. Para poder crear el Regex parser se ha tenido que investigar y entender las etiquetas para el corpus en español. Se ha usado la referencia siguiente: https://www.sketchengine.eu/spanish-freeling-part-of-speech-tagset/ . Ademas, para crear posteriormente nuestro regexparser es importante entender las etiquetas. Para hacer bien la fragmentacion, en ese caso se ha practicado con expresiones regulares. 

In [11]:
import re

tags= [tag for word, tag in tagged]
for tag in tags:
    print(tag)
    x = re.search("s?n[cp\.].*", tag)  #nombre propio o comun.
    y = re.search("s.*", tag)       #sn.e-SUJ
    z =re.search("s?[na][cp]",tag)   #anadimos el adjetivo
    print(x ,y, z)

pp1csn00
None <_sre.SRE_Match object; span=(4, 8), match='sn00'> None
rn
None None None
vmip1s0
None <_sre.SRE_Match object; span=(5, 7), match='s0'> None
di0fs0
None <_sre.SRE_Match object; span=(4, 6), match='s0'> None
ncfs000
<_sre.SRE_Match object; span=(0, 7), match='ncfs000'> <_sre.SRE_Match object; span=(3, 7), match='s000'> <_sre.SRE_Match object; span=(0, 2), match='nc'>
sps00
None <_sre.SRE_Match object; span=(0, 5), match='sps00'> None
dn0cp0
None None None
ncmp000
<_sre.SRE_Match object; span=(0, 7), match='ncmp000'> None <_sre.SRE_Match object; span=(0, 2), match='nc'>
Fc
None None None
cc
None None None
dn0cp0
None None None
ncmp000
<_sre.SRE_Match object; span=(0, 7), match='ncmp000'> None <_sre.SRE_Match object; span=(0, 2), match='nc'>
sps00
None <_sre.SRE_Match object; span=(0, 5), match='sps00'> None
ncfp000
<_sre.SRE_Match object; span=(0, 7), match='ncfp000'> None <_sre.SRE_Match object; span=(0, 2), match='nc'>


In [12]:
tags= [tag for word, tag in tagged]
for tag in tags:
    print(tag)
    x = re.search("d[in].*", tag)  #determiner : dn0cp0  indefinit 
    y = re.search("Z.*", tag)       #number
    print(x ,y)

pp1csn00
None None
rn
None None
vmip1s0
None None
di0fs0
<_sre.SRE_Match object; span=(0, 6), match='di0fs0'> None
ncfs000
None None
sps00
None None
dn0cp0
<_sre.SRE_Match object; span=(0, 6), match='dn0cp0'> None
ncmp000
None None
Fc
None None
cc
None None
dn0cp0
<_sre.SRE_Match object; span=(0, 6), match='dn0cp0'> None
ncmp000
None None
sps00
None None
ncfp000
None None


# Version1. Regex parser



In [16]:
grammar = r"""
    Comida: {<s?n[cp\.].*><s.*>?<s?[na][cp\.].*>*}
            {<s?n[cp\.].*>}
    Cantidad: {<d[in].*>}
              {<Z.*>}  
"""

regex_parser = nltk.RegexpParser(grammar)

In [17]:
sentence = 'yo quiero 3 bocadillos con cebolla, 3 pizzas y ensalada'
tokens = nltk.word_tokenize(sentence)
tagged = spanish_pos_tagger.tag(tokens)
chunked = regex_parser.parse(tagged)
print(chunked)
nltk.chunk.tree2conlltags(chunked)

(S
  yo/pp1csn00
  quiero/vmip1s0
  (Cantidad 3/di0fp0)
  (Comida bocadillos/ncfp000 con/sps00 cebolla/np0000l)
  ,/Fc
  (Cantidad 3/Z)
  (Comida pizzas/ncmp000)
  y/cc
  (Comida ensalada/sn.e-SUJ))


[('yo', 'pp1csn00', 'O'),
 ('quiero', 'vmip1s0', 'O'),
 ('3', 'di0fp0', 'B-Cantidad'),
 ('bocadillos', 'ncfp000', 'B-Comida'),
 ('con', 'sps00', 'I-Comida'),
 ('cebolla', 'np0000l', 'I-Comida'),
 (',', 'Fc', 'O'),
 ('3', 'Z', 'B-Cantidad'),
 ('pizzas', 'ncmp000', 'B-Comida'),
 ('y', 'cc', 'O'),
 ('ensalada', 'sn.e-SUJ', 'B-Comida')]

En la practica se pide mostrar la comida y la cantidad por separado. Asi que se crea una funcion llamada pedir_comida para que se muestre en forma de diccionario. 

In [39]:
 def pedir_comida(chunked_sentence) :
    pedido = []
    dict_ = dict()
    default_cant = 1 #por defecto descrita en el ejercicio
    #list comprehension. incluimos todas aquellas palabras que tengan comida o cantidad.
    chunks = [chunk for chunk in nltk.chunk.tree2conlltags(chunked_sentence) 
                  if 'Cantidad' in chunk[2] or 'Comida' in chunk[2]]
    
    for i in range(0,len(chunks)):
        chunk = chunks[i]
        word, tag, chunk_type = chunk
        
        if chunk_type == 'B-Cantidad' :
            dict_['cantidad'] = word

        if chunk_type == 'B-Comida' :
            if 'cantidad' not in dict_ :
                dict_['cantidad'] = default_cant

            dict_['comida'] = word
            pedido.append(dict_)
            dict_ = {}
        
    return pedido

In [106]:
print(sentence)
pedir_comida(chunked)

Quisiera cuatro solomillos


[{'cantidad': '3', 'comida': 'bocadillos'},
 {'cantidad': '3', 'comida': 'pizzas'},
 {'cantidad': 1, 'comida': 'ensalada'}]

# Version 2 . Unigramchunker y BigramChunker y NaiveBayesClassifier

Para los chunkers, lo primero que hay que hacer es crear un corpus en formato IOB. Para obtener el IOB tag de una nueva frase, utilizaremos el pos_tagger_spanish y el regexparser creados anteriormente. El formato IOB es el formato estandard para representar estructuras fragmentadas(chunked structures), además NLTK da metodos para leer y escribir en dicho formato. Por eso debemos hacer la conversion. 



In [60]:
def load_corpus():
    corpus_pedidos = ['Quisiera pedir un hamburguesa', 'Quiero una tortilla y una cerveza',
                     'Me pones un pollo y una ensalada', 'Quiero una paella', 'Quiero un bocadillo', 
                     'Quiero una pizza', 'Ponme una sopa', 'Quiero un filete', 'Quisiera pedir una ensalada',
                     'Quiero cinco bocadillos', 'Quisiera una empanada', 'Quiero unas croquetas', 'Quisiera morcilla', 
                     'Quiero pedir un solomillo', 'Quiero unos macarrones', 'Quiero una Lasagna', 'Quiero una hamburguesa', 
                     ' una de patatas fritas y una cerveza', 'Quiero un lenguado', 'Quiero un bonito', 'Quisiera una sepia',
                     'Quiero cinco cervezas', 'Quiero tres sidras y tres pinchos', 'Quiero cinco manzanas y tres melocotones',
                     'Quisiera cuatro solomillos', 'Quiero una naranja y dos peras', 'Quisiera pedir un hamburguesa',
                     'Me gustaría comer una tortilla de patatas', '¿Nos pones 3 tocinos, 4 pechugas?',
                     '5 repollos, 12 sardinas y 8 jalapeños',
                     'Estamos indecisos, pero puedes ponernos mientras tres raciones de patatas fritas',
                     '¿Sería tan amable de servirnos catorce tostadas?',
                     'Queremos 2 pollos con caracoles, una ternera, tres patos y 2 pimientos con arroz', 
                     'Nuestro pedido es: 10 hamburguesas con queso, 20 pizzas y 3 patatas',
                     'Yo quiero 9 yogures, 12 ensaladas, un pimiento, 4 chorizos, 12 empanadillas de atún, 8 crespillos y 3 alubias',
                     'Me gustaría comer: kiwi con ensalada', 'Él quiere pedir 3 tostadas, 2 tomates y 6 uvas',
                     'Ellos han pedido 2 higos, 1 salmón con caracoles y arroz', 
                     'Tú has pedido 4 fajitas de pollo, dos emperadores con patatas y tres rúculas', 
                     '2 macarrones, 3 quesos, una guindilla, cuatro pulpos y quince verduras', 
                     'Mi abuela quiere emperador con ensalada', 'Mi tía quiere comer marisco y carne',
                     '2 cebollas, una calabaza y ocho tomates', 'tres pollos, dos corderos, cuatro cerdos y tres macarrones',
                     'Pepinillos, calabaza, navajas, sepia y gulas', 'Me gustaría comer bonito a la plancha con patatas asadas', 
                     'Pídete cinco pomelos, tres calabacines, 8 mangos, seis melocotonesLangosta, langostinos y almejas']
      
    return corpus_pedidos

len(load_corpus())

47

In [32]:
def iob_corpus(corpus, regex_parser) :
    iob_corpus_pedidos = []
    for sent in corpus :
        tokens = nltk.word_tokenize(sent)
        tagged_sentence = spanish_pos_tagger.tag(tokens)
        chunked_sentence = regex_parser.parse(tagged_sentence)
        iob_corpus_pedidos.append(chunked_sentence)
        
    return iob_corpus_pedidos

In [33]:
iob_corpus = iob_corpus(load_corpus(), regex_parser)
#print(iob_tags)

Ahora que tenemos las etiquetas en formato iob, podremos seguir los pasos del capítulo 7 del NLTK book. Usando las clases de Unigram y BigramChunker primero.

In [34]:
class UnigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.UnigramTagger(train_data) 

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

In [59]:
class BigramChunker(nltk.ChunkParserI):
    def __init__(self, train_sents): 
        train_data = [[(t,c) for w,t,c in nltk.chunk.tree2conlltags(sent)]
                      for sent in train_sents]
        self.tagger = nltk.BigramTagger(train_data) 

    def parse(self, sentence):
        pos_tags = [pos for (word,pos) in sentence]
        tagged_pos_tags = self.tagger.tag(pos_tags)
        chunktags = [chunktag for (pos, chunktag) in tagged_pos_tags]
        conlltags = [(word, pos, chunktag) for ((word,pos),chunktag)
                     in zip(sentence, chunktags)]
        return nltk.chunk.conlltags2tree(conlltags)

Para evaluar los chunkers, tenemos que dividir nuestro corpus como hemos hecho anteriormente con el cess_esp. el corpus tiene 47 frases. 

In [61]:
training = iob_corpus[0:40]
testing = iob_corpus[40:]

Podemos evaluar el UnigramChunker y los datos test. 

In [62]:
unigram_chunker = UnigramChunker(training)
print(unigram_chunker.evaluate(testing))

sentence = random.choice(load_corpus())
tokens = nltk.word_tokenize(sentence)
tagged = spanish_pos_tagger.tag(tokens)

print()
print(sentence)
print(pedir_comida(unigram_chunker.parse(tagged)))

ChunkParse score:
    IOB Accuracy:  95.5%%
    Precision:     90.9%%
    Recall:        93.8%%
    F-Measure:     92.3%%

Quisiera pedir una ensalada
[{'cantidad': 'una', 'comida': 'ensalada'}]


In [63]:
bigram_chunker = BigramChunker(training)
print(bigram_chunker.evaluate(testing))
sentence=random.choice(load_corpus())
tokens = nltk.word_tokenize(sentence)
tagged = spanish_pos_tagger.tag(tokens)

print()
print(sentence)
print(pedir_comida(bigram_chunker.parse(tagged)))

ChunkParse score:
    IOB Accuracy:  76.1%%
    Precision:    100.0%%
    Recall:        59.4%%
    F-Measure:     74.5%%

Me pones un pollo y una ensalada
[{'cantidad': 'un', 'comida': 'pollo'}, {'cantidad': 'una', 'comida': 'ensalada'}]


Para observar diferentes Chunkers, podemos observar y evaluar el NaiveBayesChunker. En el apartado 3.3 del capitulo 7 , se explica la formacion de chunkers basados en clasificadores. Para maximizar el rendimiento de la fragmentacion, necesitamos informacion del contenido de las palabras a parte de sus etiquetas. Por ese motivo usamos ConsecutivePosTagger y NaiveBayesClassifier. 


In [83]:
class ConsecutivePosTagger(nltk.TaggerI):

    def __init__(self, train_sents):
        train_set = []
        for tagged_sent in train_sents:
            untagged_sent = nltk.tag.untag(tagged_sent)
            history = []
            for i, (word, tag) in enumerate(tagged_sent):
                featureset = npchunk_features(untagged_sent, i, history) 
                train_set.append( (featureset, tag) )
                history.append(tag)
                
        self.classifier = nltk.NaiveBayesClassifier.train(train_set)

    def tag(self, sentence):
        history = []
        for i, word in enumerate(sentence):
            featureset = npchunk_features(sentence, i, history)
            tag = self.classifier.classify(featureset)
            history.append(tag)
        return zip(sentence, history)

    
    
class NaiveBayesChunker(nltk.ChunkParserI): 
    def __init__(self, train_sents):
        tagged_sents = [[((w,t),c) for (w,t,c) in
                         nltk.chunk.tree2conlltags(sent)]
                        for sent in train_sents]
        self.tagger = ConsecutivePosTagger(tagged_sents)

    def parse(self, sentence):
        tagged_sents = self.tagger.tag(sentence)
        conlltags = [(w,t,c) for ((w,t),c) in tagged_sents]
        return nltk.chunk.conlltags2tree(conlltags)

In [84]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    return {"pos": pos}

In [97]:
naive_chunker = NaiveBayesChunker(training)
print(naive_chunker.evaluate(testing))

sentence = random.choice(load_corpus())
tokens = nltk.word_tokenize(sentence)
tagged = spanish_pos_tagger.tag(tokens)

parsed = naive_chunker.parse(tagged)

print()
print(sentence)
print()
print(pedir_comida(parsed))

ChunkParse score:
    IOB Accuracy:  89.6%%
    Precision:     81.8%%
    Recall:        84.4%%
    F-Measure:     83.1%%

tres pollos, dos corderos, cuatro cerdos y tres macarrones

[{'cantidad': 'tres', 'comida': 'pollos'}, {'cantidad': 'dos', 'comida': 'corderos'}, {'cantidad': 'cuatro', 'comida': 'cerdos'}, {'cantidad': 'tres', 'comida': 'macarrones'}]


In [101]:
def npchunk_features(sentence, i, history):
    word, pos = sentence[i]
    
    if i == 0:
        prevword, prevpos = "<START>", "<START>"
    else:
        prevword, prevpos = sentence[i-1]
    
    return { " pos " : pos, " word " : word,  " prevpos " : prevpos}

In [104]:
naive_chunker = NaiveBayesChunker(training)
print(naive_chunker.evaluate(testing))

sentence = random.choice(load_corpus())
tokens = nltk.word_tokenize(sentence)
tagged = spanish_pos_tagger.tag(tokens)

parsed = naive_chunker.parse(tagged)

print()
print(sentence)
print()
print(pedir_comida(parsed))

ChunkParse score:
    IOB Accuracy:  86.6%%
    Precision:     75.8%%
    Recall:        78.1%%
    F-Measure:     76.9%%

Quisiera cuatro solomillos

[{'cantidad': 'cuatro', 'comida': 'solomillos'}]


In [105]:
print("UnigramChunker")
print(unigram_chunker.evaluate(testing))
print("BigramChunker")
print(bigram_chunker.evaluate(testing))
naive_chunker = NaiveBayesChunker(training)
print("NaiveBayesChunker (token actual como unigram chunker)")
print(naive_chunker.evaluate(testing))
naive_chunker = NaiveBayesChunker(training)
print("NaiveBayesChunker(token actual y previo )")
print(naive_chunker.evaluate(testing))

UnigramChunker
ChunkParse score:
    IOB Accuracy:  95.5%%
    Precision:     90.9%%
    Recall:        93.8%%
    F-Measure:     92.3%%
BigramChunker
ChunkParse score:
    IOB Accuracy:  76.1%%
    Precision:    100.0%%
    Recall:        59.4%%
    F-Measure:     74.5%%
NaiveBayesChunker (token actual como unigram chunker)
ChunkParse score:
    IOB Accuracy:  86.6%%
    Precision:     75.8%%
    Recall:        78.1%%
    F-Measure:     76.9%%
NaiveBayesChunker(token actual y previo )
ChunkParse score:
    IOB Accuracy:  86.6%%
    Precision:     75.8%%
    Recall:        78.1%%
    F-Measure:     76.9%%


Con las evaluaciones anteriores podemos observar que UnigramChunker y NaiveBayeschunker son los mejores para nuestro corpus de entrenamiento. Es cierto que nuestro corpus al ser pequeño pude crear sobreajuste y estas evaluaciones no son fiables.

Para el NaiveBayesChunker tenemos que recordar que usando el POS tag previo, estamos usando el teorema de bayes y multiplicando la probabilidad de palabras conjuntas. Al tener un corpus_pedido pequeño y poco variado, no podemos dar  por validos los resultados. En teoria, NaiveBayesChunker es el mejor. 

Para el Bigramtagger pasa lo mismo. Si miramos las etiquetas anteriores con tan pocos ejemplos, no los podemos entrenar debidamente. 

Podemos concluir que para esta practica nos quedariamos con Unigram o Naivebayes. Siempre recordando que hemos usado un corpus de variedad my escaso y pequeño.
